In [ ]:
from typing import List, Dict, Tuple, Optional, Union, Any

import pandas as pd

from prompt_template_collection import PromptTemplate
from doraemon import Doraemon
from relaxed_fda import RelaxedFDA
from metrics_collection import MetricsHelper

task_name = "HotpotQA"
# Set up logger
logger = Doraemon.get_logger(name=task_name, logfile=f"cp_on_{task_name}.log")

# Prepare dataset with the logger passed as a parameter
file_path = '/kaggle/input/building-sots-on-hotpotqa-injected/sots_df.pkl'
grouped_data = RelaxedFDA.prepare_dataset(file_path=file_path, mode='r_s', logger=logger)

encoder=RelaxedFDA.get_encoder()

# Demo promts question, corect, and wrong 
D: List[Dict] = PromptTemplate.sot_construct_inter_hotpotqa()
logger.info(len(D))

op_system_prompt = RelaxedFDA.get_optimize_system_prompt(task_name)
logger.info(op_system_prompt)

In [2]:
grouped_data[['question', 'context']] = grouped_data['question'].str.split(r'\nContext:', n=1, expand=True)
grouped_data['question']=grouped_data['question'].str.strip()
grouped_data['context']=grouped_data['context'].str.strip()

In [ ]:
result_pd: pd.DataFrame = RelaxedFDA.evaluate(
    grouped_data,
    logger,
    op_system_prompt,
    encoder,
    D,
    enable_logger_rs=False
)

In [ ]:
MetricsHelper.evaluate(result_pd, logger)